#### initialize AI Platform 

```python
aiplatform.init(project=PROJECT_ID, location=REGION)
```

### Create predictions without doing a deployment 

```python
#@title Select your input parameters { run: "auto", display-mode: "form" }
mintemp = 14 #@param {type:"slider", min:1, max:90, step:1}
maxtemp = 93 #@param {type:"slider", min:10, max:110, step:1}
rain = 0 #@param {type:"slider", min:0, max:5, step:0.1}
dayofweek = 4 #@param {type:"slider", min:1, max:7, step:1}

query = """
  select *
  from ml.PREDICT(
    MODEL {0}.{1},  (
      select  '{5}' as dayofweek,
              {2} as mintemp,
              {3} as maxtemp,
              {4} as rain
    )
  )
""".format(BQ_DATASET_NAME,
           BQML_MODEL_NAME,
           mintemp,
           maxtemp,
           rain,
           dayofweek)

df = bq_client.query(query).to_dataframe()
df["predicted_label"] = np.multiply(df["predicted_label"],maxtrips).astype(int)
df
```

# Datasets

```python
# Create dataset if it doesn't exist
datasets = aiplatform.TextDataset.list(filter = f"display_name={dataset_display_name}")

if datasets:
    text_dataset = datasets[0]
    print(f"Dataset Exists: {datasets[0].resource_name}")
else:
    text_dataset = aiplatform.TextDataset.create(
    display_name=dataset_display_name,
    gcs_source=dataset_source_public,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
    sync=True,
    )
    print(f"Dataset Created: {text_dataset.resource_name}")
    
print(f'Review the Dataset in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/datasets/{datasets[0].resource_name}?project={PROJECT_ID}')


```

# Models

```python

#
# Create Model 
#

# Create model job if it doesn't exist
models = aiplatform.Model.list(filter = f"display_name={model_display_name}")

if models:
    model = models[0]
    print(f"Model Exists: {models[0].resource_name}")
else:
    model = job.run(
    dataset=text_dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.1,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    sync=True,
    )

print(f"Model Created: {model.resource_name}")


#
# Create
# 

create_model = True
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        create_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name

else:
    print('This is a new model, creating in model registry')
    parent_model = ''

if create_model:
    model = aiplatform.Model.create(
        display_name = f'{model_display_name}_{EXPERIMENT}',
        model_id = f'model_{model_display_name}_{EXPERIMENT}',
        parent_model =  modelmatch[0].resource_name,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{model_display_name}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
    )


#
# Upload Model 
#

upload_model = True
if modelmatch:
    print("Model Already in Registry:")
    if RUN_NAME in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        upload_model = False
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        parent_model = modelmatch[0].resource_name

else:
    print('This is a new model, creating in model registry')
    parent_model = ''

if upload_model:
    model = aiplatform.Model.upload(
        display_name = f'{model_display_name}_{EXPERIMENT}',
        model_id = f'model_{model_display_name}_{EXPERIMENT}',
        parent_model =  modelmatch[0].resource_name,
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/models/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [RUN_NAME],
        version_description = RUN_NAME,
        labels = {'series' : f'{model_display_name}', 'experiment' : f'{EXPERIMENT}', 'experiment_name' : f'{EXPERIMENT_NAME}', 'run_name' : f'{RUN_NAME}'}        
    )
```

# Endpoints

```python

### Create/Retrieve The Endpoint For This Series

endpoints = aiplatform.Endpoint.list(filter = f"labels.series={model_display_name}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{model_display_name}",
        labels = {'series' : f"{model_display_name}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")
    
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

```

# Jobs

```python
# Define the training job and create one if it doesn't exit 
jobs = aiplatform.AutoMLTextTrainingJob.list(filter = f"display_name={job_display_name}")

if jobs:
    job = jobs[0]
    print(f"Jobs Exists: {jobs[0].resource_name}")
else:
    job = aiplatform.AutoMLTextTrainingJob(
    display_name=job_display_name,
    prediction_type="classification",
    multi_label=False,
    )
    print(f"job Created: {job.resource_name}")
    
print(f'Review the job in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/jobs/{jobs[0].resource_name}?project={PROJECT_ID}')

```

print(job)

# Experiments

```python
## Initialize Experiment 
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

## Get a List of all your experiments
experiments = aiplatform.Experiment.list()

## Check if the Experiment exists 
experiment = aiplatform.Experiment.list(filter=f"labels.series={experiment_name}")

if experiment:
    experiment = experiment[0]
else: 
    experiment = aiplatform.Experiment.create(display_name = model_display_name, labels = {'series' : f'{model_display_name}'})

```

# TensorBoard
```python

tb = aiplatform.Tensorboard.list(filter=f"labels.series={model_display_name}")

if tb:
    tb = tb[0]
else: 
    tb = aiplatform.Tensorboard.create(display_name = model_display_name, labels = {'series' : f'{model_display_name}'})

```